# Homework 3:  Creating Fire Smoke Estimates of Wildfires in Centennial, Colarado

## Part 1: Common Analysis

More and more frequently summers in the western US have been characterized by wildfires with smoke billowing across multiple western states. There are many proposed causes for this: climate change, US Forestry policy, growing awareness, just to name a few. Regardless of the cause, the impact of wildland fires is widespread as wildfire smoke reduces the air quality of many cities. There is a growing body of work pointing to the negative impacts of smoke on health, tourism, property, and other aspects of society.

In this project I analyzed wildfire impacts in **Centennial, Colarado** in the US. The end goal is to be able to inform policy makers, city managers, city councils, or other civic institutions, to make an informed plan for how they could or whether they should make plans to mitigate future impacts from wildfires.


### License

This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - August 16, 2024

The following functions were used from Dr. David W. McDonald's Python Notebook File `wildfire_geo_proximity_example.ipynb`:  
1. `convert_ring_to_epsg4326`
2. `shortest_distance_from_place_to_fire_perimeter`

The rest of the code lies under the standard MIT license.

#### External Python Libraries
This notebook has dependencies on [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html), the [geojson](https://pypi.org/project/geojson/) module and on the wildfire user module. `Pyproj`, `geojson` and `pandas` can be installed via `pip`.


####  External Python Libraries
This notebook requires external Python libraries, including:
- **[Pyproj](https://pyproj4.github.io/pyproj/stable/index.html)**: For geographic transformations and distance calculations.
- **[GeoJson](https://pypi.org/project/geojson/)**: For handling GeoJSON format files, a common format for spatial data.
- **Pandas**: For data manipulation and analysis.

Additional standard libraries used:
- `os`, `json`, `time`, and `datetime` for file handling and date manipulation.
- **Wildfire user module**: Provides tools for reading GeoJSON files of wildfire data.

Install these dependencies by uncommenting the below cell

In [27]:
# Uncomment the below lines of code to install the required python libraries
# !pip install pyproj
# !pip install geojson
# !pip install pandas

In [1]:
#
#    IMPORTS
# 
#    Import some standard python modules
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of wildfires extracted from the main wildfire dataset.
#
import geojson

# Other generic imports
from datetime import datetime
import pandas as pd

## Step 0: Data Acquisation

The common analysis research question is based on one specific dataset. You should get the [Combined wildland fire datasets for the United States and certain territories](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81), [1800s-Present (combined wildland fire polygons)](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) dataset. This dataset was collected and aggregated by the US Geological Survey. The dataset is relatively well documented. The dataset provides fire polygons in ArcGIS and GeoJSON formats. 


### Details about Centennial, Colarado


| City       | ST | 2023 Estimate | 2020 Census | 2020 Density (mi²) | Location         |
|------------|----|---------------|-------------|---------------------|-------------------|
| Centennial | CO | 106,883       | 108,418     | 3,650               | 39.59°N 104.87°W  |


In the dataset, I utilized decimal latitude and longitude coordinates sourced from the [GeoHack](https://geohack.toolforge.org/geohack.php?pagename=List_of_United_States_cities_by_population&params=39.59_N_104.87_W_&title=Centennial) website.


In [10]:
#
#    CONSTANTS
#
DATA_FILE = "data/USGS_Wildland_Fire_Combined_Dataset.json"

CITY_LOCATION = {
    'city'   : 'Centennial',
    'latlon' : [39.59, -104.87] 
}


## Load the wildfire data using the GeoJSON module

In this example we use the GeoJSON module ([documentation](https://pypi.org/project/geojson/), [GitHub repo](https://github.com/jazzband/geojson)) to load the sample file. This module works mostly the way you would expect. This examples does not rely on specific Geo features from `geojson`.

The JSON object has a total of 135061 features. Loading the entire dataset as a `json` file took minutes 4min 14s on a laptop with a 13th Gen Intel Core i5-1335U processor and 16GB of RAM.

In [3]:
%%time


try:
    # Load only the essential 'features' field from the GeoJSON file.
    print(f"Loading 'features' from GeoJSON file: '{DATA_FILE}'")
    with open(DATA_FILE, "r") as geojson_file:
        geojson_data = json.load(geojson_file)
        features = geojson_data.get('features', [])
    
    # Output the keys and feature count to confirm data loading.
    print("Loaded GeoJSON data keys:", list(geojson_data.keys()))
    print(f"Total features in GeoJSON data: {len(features)}")
    print()

except FileNotFoundError:
    print(f"Error: File '{DATA_FILE}' not found.")

Loading 'features' from GeoJSON file: 'data/USGS_Wildland_Fire_Combined_Dataset.json'
Loaded GeoJSON data keys: ['displayFieldName', 'fieldAliases', 'geometryType', 'spatialReference', 'fields', 'features']
Total features in GeoJSON data: 135061

CPU times: total: 1min 53s
Wall time: 4min 14s


This code calculates and filters wildfires that have occurred in the last 60 years. It begins by determining the cutoff year (60 years before the current year) and then filters wildfire data from a GeoJSON structure, selecting only records where the year of occurrence is within this timeframe. The filtered list is stored in `recent_wildfires`, which contains only recent wildfire events. Finally, the code prints the total count of these filtered wildfires and displays a sample entry, providing insight into the structure of the data.

In [4]:
# Define the starting year as 60 years ago from the current year.
current_year = datetime.now().year
cutoff_year = current_year - 60

# Filter wildfires based on the cutoff year, keeping only those from the last 60 years.
recent_wildfires = [fire for fire in geojson_data['features'] if fire['attributes']["Fire_Year"] >= cutoff_year]

# Print the total number of wildfires that occurred in the last 60 years.
print(f"Total number of wildfires since {cutoff_year}: {len(recent_wildfires)}")

# Display a sample wildfire from the filtered results.
print("Sample wildfire from the filtered list:")
print(json.dumps(recent_wildfires[0], indent=4))


Total number of wildfires since 1964: 117191
Sample wildfire from the filtered list:
{
    "attributes": {
        "OBJECTID": 14600,
        "USGS_Assigned_ID": 14600,
        "Assigned_Fire_Type": "Wildfire",
        "Fire_Year": 1964,
        "Fire_Polygon_Tier": 1,
        "Fire_Attribute_Tiers": "1 (1), 3 (3)",
        "GIS_Acres": 65338.8776355638,
        "GIS_Hectares": 26441.70565918891,
        "Source_Datasets": "Comb_National_NIFC_Interagency_Fire_Perimeter_History (1), Comb_National_WFDSS_Interagency_Fire_Perimeter_History (1), Comb_State_California_Wildfire_Polygons (1), Comb_National_USFS_Final_Fire_Perimeter (1)",
        "Listed_Fire_Types": "Wildfire (3), Likely Wildfire (1)",
        "Listed_Fire_Names": "COYOTE (4)",
        "Listed_Fire_Codes": "No code provided (4)",
        "Listed_Fire_IDs": "0 (3)",
        "Listed_Fire_IRWIN_IDs": "",
        "Listed_Fire_Dates": "Listed Wildfire Discovery Date(s): 1964-09-22 (2) | Listed Other Fire Date(s): 1964-09-22 - DATE_

## Distance Computations with Pyproj

When performing geodetic calculations, we encounter challenges due to the Earth's shape. Geographic coordinate systems are projected onto the Earth, which is not flat but an ellipsoid with varying curvature depending on location and direction. This means that distance calculations between two points on Earth's surface represent an arc rather than a straight line. 

A common requirement in geodetic computations is ensuring that all points are in the same geographic coordinate system. Many coordinate systems exist, each suited to specific uses and regions, and they are documented at resources like [EPSG.io](https://epsg.io).

In our wildfire dataset, we see fields such as `"geometryType"` and `"spatialReference"`, shown as:
```json
"geometryType": "esriGeometryPolygon",
"spatialReference": {
    "wkid": 102008,
    "latestWkid": 102008
}
```
This indicates that wildfire geometries are polygons in the coordinate system identified by WKID 102008, also known as [ESRI:102008](https://epsg.io/102008).

The coordinate system commonly used for decimal degrees (often retrieved from sources like Google) is [EPSG:4326](https://epsg.io/4326), known as *WGS84*. For consistency in our example, we will convert fire boundary points from ESRI:102008 to EPSG:4326. This allows us to align fire data coordinates with other latitude and longitude values in decimal degrees, facilitating distance calculations between them.

## Calculating Distance from Centennial, CO to a Wildfire

The challenge is to determine the distance between a fire and a specific location, such as Centennial, CO. Fires have irregular shapes, so defining "distance" to them depends on how we conceptualize it. For instance, we might identify the closest point on the fire's perimeter to Centennial and use that as the distance, or we might find the fire's centroid (center point) and calculate the distance to that. Many other approaches are also possible.

_We use the following method to estimate this distance_   
**Shortest Perimeter Distance**: The first approach finds the closest point on the fire's perimeter to Centennial and returns this minimum distance, along with the latitude and longitude of that perimeter point.

I used the shortest perimeter distance instead of the average distance because it directly indicates the closest point on the fire's perimeter. Additionally, the calculation of the shortest distance is typically simpler and more straightforward. Overall, I think that the shortest distance offers a more accurate representation for my usecase.

My function takes a list of coordinates in the ESRI:102008 coordinate system and converts them into the EPSG:4326 format (using decimal degrees for latitude and longitude). After that, I calculated the distance from the specified location to each point on the transformed fire perimeter using the WGS84 ellipsoid model, keeping track of the shortest distance found and the nearest point on the perimeter. Finally, it returns this information as a list containing the closest distance and the corresponding perimeter point.

In [6]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

In [7]:
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point


This code processes a list of recent wildfire data to calculate and store the closest distance from Centennial, CO to each wildfire's perimeter. For each wildfire entry, it retrieves the geometry type, either rings or curveRings, and extracts the perimeter coordinates accordingly. Using these coordinates, the code calculates the shortest distance to the fire's boundary. The computed distance and selected wildfire attributes are then saved to `wildfire_list`. If any entry lacks compatible geometry, an error is raised, and any calculation issues are logged for further debugging.

There are 28 values for which we have an irregular geometry shape and I have decided to ignore those values in this dataset. 

This distance computation takes around 30 mins on a laptop with a 13th Gen Intel Core i5-1335U processor and 16GB of RAM the first time I did it but for this run it shows an estimate of 8h 22min 33s.

In [12]:
%%time
wildfire_list = list()
fields_of_interest = ['OBJECTID', 
                      'USGS_Assigned_ID', 
                      'Assigned_Fire_Type', 
                      'Fire_Year', 
                      'Fire_Polygon_Tier', 
                      'Fire_Attribute_Tiers', 
                      'GIS_Acres', 
                      'GIS_Hectares', 
#                       'Source_Datasets', 
                      'Listed_Fire_Types', 
                      'Listed_Fire_Names', 
                      'Listed_Fire_Codes', 
                      'Listed_Fire_IDs', 
                      'Listed_Fire_IRWIN_IDs', 
                      'Listed_Fire_Dates', 
                      'Listed_Fire_Causes', 
                      'Listed_Fire_Cause_Class', 
                      'Listed_Rx_Reported_Acres', 
#                       'Listed_Map_Digitize_Methods', 
                    #   'Listed_Notes', 
                    #   'Processing_Notes', 
                    #   'Wildfire_Notice', 
                    #   'Prescribed_Burn_Notice', 
                      'Wildfire_and_Rx_Flag', 
                      'Overlap_Within_1_or_2_Flag', 
                      'Circleness_Scale', 
                      'Circle_Flag', 
#                       'Exclude_From_Summary_Rasters', 
                      'Shape_Length', 
                      'Shape_Area']


for i, fire in enumerate(recent_wildfires):

    fire_info = fire['attributes']
    fire_shape = fire['geometry']

    # Identify and assign the perimeter coordinates based on available geometry type.
    if 'rings' in fire_shape:
        perimeter_coords = fire_shape['rings'][0]
    elif 'curveRings' in fire_shape:
        perimeter_coords = fire_shape['curveRings'][0]
    else:
        raise Exception("Error: Unsupported geometry type in wildfire data.")

    try:
        # Calculate the shortest distance to any point on the wildfire's perimeter.
        min_distance = shortest_distance_from_place_to_fire_perimeter(CITY_LOCATION['latlon'], perimeter_coords)

        # Collect relevant wildfire attributes and add computed distance.
        wildfire_record = {key: fire_info[key] for key in fields_of_interest}
        wildfire_record["Min_Distance_Miles"] = min_distance[0]

        wildfire_list.append(wildfire_record)
    except Exception as ex:
        print(f"Error processing wildfire at index {i}, ID: {fire_info['USGS_Assigned_ID']}")
        print(ex)


Error processing wildfire at index 91887, ID: 109605
0
Error processing wildfire at index 92506, ID: 110224
0
Error processing wildfire at index 92921, ID: 110639
0
Error processing wildfire at index 93713, ID: 111431
0
Error processing wildfire at index 94179, ID: 111897
0
Error processing wildfire at index 94692, ID: 112410
0
Error processing wildfire at index 94697, ID: 112415
0
Error processing wildfire at index 95693, ID: 113411
0
Error processing wildfire at index 95947, ID: 113665
0
Error processing wildfire at index 96020, ID: 113738
0
Error processing wildfire at index 96048, ID: 113766
0
Error processing wildfire at index 96087, ID: 113805
0
Error processing wildfire at index 96591, ID: 114309
0
Error processing wildfire at index 96604, ID: 114322
0
Error processing wildfire at index 97911, ID: 115629
0
Error processing wildfire at index 98256, ID: 115974
0
Error processing wildfire at index 98517, ID: 116235
0
Error processing wildfire at index 99368, ID: 117086
0
Error proc

The data generated will reside as a CSV file in the location `intermediate_files/wildfires_with_distances.csv` for future reference.

In [14]:
wildfires_df_pandas = pd.DataFrame(wildfire_list)
wildfires_df_pandas.to_csv("intermediate_files/wildfires_with_distances.csv")